In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="MQckwlbA2C8gCA9kMwlh")
project = rf.workspace("anu-univ").project("pest-c1x15")
version = project.version(1)
dataset = version.download("yolov9")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 56.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to pest-1 in yolov9:: 100%|██████████| 14130/14130 [00:11<00:00, 1220.79it/s]


In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [ ]:
yaml_content = """names:
- Anoplophora_chinensis
- Apriona_germari-Hope-
- Cerambycidae_larvae
- Chalcophora_japonica
- Clostera_anachoreta
- Cnidocampa_flavescens-Walker-
- Cnidocampa_flavescens-Walker_pupa-
- Drosicha_contrahens_female
- Drosicha_contrahens_male
- Erthesina_fullo
- Erthesina_fullo_nymph
- Erthesina_fullo_nymph-2
- Hyphantria_cunea
- Hyphantria_cunea_larvae
- Hyphantria_cunea_pupa
- Latoia_consocia_Walker
- Latoia_consocia_Walker_larvae
- Micromelalopha_troglodyta-Graeser-
- Micromelalopha_troglodyta-Graeser-_larvae
- Monochamus_alternatus
- Plagiodera_versicolora-Laicharting-
- Plagiodera_versicolora-Laicharting-_larvae
- Plagiodera_versicolora-Laicharting-_ovum
- Psacothea_hilaris-Pascoe-
- Psilogramma_menephron
- Psilogramma_menephron_larvae
- Sericinus_montela
- Sericinus_montela_larvae
- Spilarctia_subcarnea-Walker-
- Spilarctia_subcarnea-Walker-_larvae
- Spilarctia_subcarnea-Walker-_larvae-2
nc: 31
roboflow:
  license: MIT
  project: pest-c1x15
  url: https://universe.roboflow.com/anu-univ/pest-c1x15/dataset/1
  version: 1
  workspace: anu-univ
test: /content/pest-1/test/images
train: /content/pest-1/train/images
val: /content/pest-1/valid/images

"""

# Define file path
file_path = "/content/pest-1/data.yaml"

# Write to file
with open(file_path, "w") as file:
    file.write(yaml_content)

print(f"File saved successfully at {file_path}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO

# Load the YOLOv5 model
model = YOLO('yolov9s.pt')  # You can use 'yolov5m.pt', 'yolov5l.pt', 'yolov5x.pt' depending on the size

# Train the model
model.train(data='/content/pest-1/data.yaml', epochs=15, imgsz=640)


In [ ]:
import subprocess
subprocess.run(["pip", "install", "gradio"])



In [ ]:
import gradio as gr
import torch
from ultralytics import YOLO
from PIL import Image, ImageDraw
import numpy as np

# Load YOLO model
model_path = "/content/runs/detect/train2/weights/best.pt"  # Update with your model path
model = YOLO(model_path)
CLASS_NAMES = model.names  # Get class names from model

# Function to make predictions
def predict(image):
    image = image.convert("RGB")
    results = model(image)  # Run inference
    pred_boxes = results[0].boxes.xyxy.cpu().numpy()
    pred_scores = results[0].boxes.conf.cpu().numpy()
    pred_labels = results[0].boxes.cls.cpu().numpy().astype(int)

    # Draw bounding boxes
    draw = ImageDraw.Draw(image)

    try:
        font = ImageFont.load_default()  # Use default font
    except:
        font = None  # Some environments may not have PIL fonts available

    for box, label, score in zip(pred_boxes, pred_labels, pred_scores):
        x_min, y_min, x_max, y_max = box
        draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=3)
        text = f"{CLASS_NAMES[label]}: {score * 100:.2f}%"  # Accuracy as percentage

        # Ensure text doesn't go out of bounds
        text_x, text_y = max(0, x_min), max(0, y_min - 10)

        # Draw text
        draw.text((text_x, text_y), text, fill="blue", font=font)

    return image

# Create Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Image(type="pil"),
    title="YOLOv5 pest Detection",
    description="Upload an image to detect pest using YOLOv5.",
)

# Launch the app
iface.launch(debug=True)


In [ ]:
import gradio as gr
import torch
import cv2
import tempfile
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw

# Load YOLO model
model_path = "/content/runs/detect/train2/weights/best.pt"  # Update with your model path
model = YOLO(model_path)
CLASS_NAMES = model.names  # Get class names from model

# Function to process video and draw bounding boxes
def predict_video(video_path):
    # Read video
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Temporary file for output video
    temp_out = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(temp_out.name, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit loop if no more frames

        # Convert frame to PIL image
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Run YOLO inference
        results = model(image)
        pred_boxes = results[0].boxes.xyxy.cpu().numpy()
        pred_scores = results[0].boxes.conf.cpu().numpy()
        pred_labels = results[0].boxes.cls.cpu().numpy().astype(int)

        # Draw bounding boxes on frame
        draw = ImageDraw.Draw(image)
        for box, label, score in zip(pred_boxes, pred_labels, pred_scores):
            x_min, y_min, x_max, y_max = box
            draw.rectangle([x_min, y_min, x_max, y_max], outline="yellow", width=5)
            text = f"{CLASS_NAMES[label]}: {score:.2f}"
            try:
                font = ImageFont.truetype("arial.ttf", 20)  # Increase font size
            except:
                font = None
            draw.text((x_min, y_min - 30), text, fill="yellow",font=font)

        # Convert image back to OpenCV format
        frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

        # Write processed frame to video
        out.write(frame)

    # Release resources
    cap.release()
    out.release()

    return temp_out.name  # Return the path of the processed video

# Create Gradio interface
iface = gr.Interface(
    fn=predict_video,
    inputs=gr.Video(),  # ✅ FIXED: Removed type="file"
    outputs=gr.Video(),
    title="YOLOv5 pest Detection (Video)",
    description="Upload a video to detect pest using YOLOv5."
)

# Launch the app
iface.launch(debug=True)
